In [1]:
import itertools
import spacy

import phunspell
import os

from spacy.matcher import Matcher
from nltk import ngrams
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import silhouette_score

import string
import re
import pandas as pd
import csv
from fuzzywuzzy import fuzz

import enchant

import numpy as np

C:\Users\sri.sai.praveen.gadi\.virtualenvs\xxx-eAHl9Gue\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\sri.sai.praveen.gadi\.virtualenvs\xxx-eAHl9Gue\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
import sklearn.metrics as metrics

In [3]:
a = [1, 0]
b = [-1, 0]

cosine_similarity(np.array(a).reshape(1, -1), np.array(b).reshape(1, -1))[0][0]

-1.0

In [2]:
nlp_de = spacy.load("de_core_news_sm")
nlp_en = spacy.load("en_core_web_sm")

C:\Users\sri.sai.praveen.gadi\.virtualenvs\xxx-eAHl9Gue\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'de_core_news_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.4.3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
C:\Users\sri.sai.praveen.gadi\.virtualenvs\xxx-eAHl9Gue\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.4.3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
r = [i for i in range(10)]
r

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [14]:
print(len(list(itertools.permutations(r[:3]))))
print(len(list(itertools.permutations(r[:5]))))
print(len(list(itertools.permutations(r[:6]))))
print(len(list(itertools.permutations(r[:7]))))
print(len(list(itertools.permutations(r[:8]))))
print(len(list(itertools.permutations(r[:9]))))
print(len(list(itertools.permutations(r[:10]))))

6
120
720
5040
40320
362880
3628800


In [12]:
np.random.permutation(np.array(r))

array([8, 3, 5, 1, 0, 4, 9, 6, 7, 2])

In [3]:
Result = [("c1", 8, 2), ("c2", 6, 2), ("c3", 5, 0), ("c4", 5, 0), ("c5", 6, 0)]

In [4]:
def calculate_expectation(Result, n_results):
    
    all_combinations = list(itertools.permutations(Result))
    list_of_found_relevant_doc_per_permutation = []
    
    for comb in all_combinations:
        
        number_of_found_relevant_docs = 0
        number_of_inspected_docs = 0
        
        for cluster in comb:
            if number_of_inspected_docs > n_results:
                break
            else:
                number_of_inspected_docs += cluster[1]
                number_of_found_relevant_docs += cluster[2]
                
        list_of_found_relevant_doc_per_permutation.append(number_of_found_relevant_docs)
        
    return sum(list_of_found_relevant_doc_per_permutation) / len(list_of_found_relevant_doc_per_permutation)

In [11]:
calculate_expectation(Result, n_results=20)

3.2

In [58]:
# for comb in itertools.permutations(Result):
# #     print(comb)

In [5]:
list(itertools.permutations(Result))
print(len(list(itertools.permutations(Result))))

120


## Noun-chunk filtering

1. Spacy noun-chunks containing 2 words
2. N-grams range (1, 2)

In [2]:
pattern1 = r"[P]+[C]*[N]*[P]*"
pattern2 = "PPNUNPDPXDP"
tag_dict = {"PROPN":"P", 
            "NOUN":"P", 
            "X":"P", 
            "VERB":"V", 
            "ADJ":"A", 
            "PUNCT": "U",
            "ADV": "B",
            "DET":"D",
            "NUM":"N",
            "CONJ":"C"}

In [3]:
 def get_ncs(sent, pattern): 
    
    doc = nlp_de(sent)
    tag_str = ""
    for tok in doc:
        tag_str = tag_str + tag_dict.get(tok.pos_, "X")
    
    result = re.findall(pattern, tag_str)
    ncs = []

    for m in result:

        start_index_token = tag_str.find(m)
        span = doc[start_index_token: start_index_token + len(m)]
        ncs.append(span.text)
        
        tag_str = tag_str[start_index_token + len(m):]
        doc = doc [start_index_token + len(m):]
        
    return ncs

In [4]:
def portion_of_capital_letters(w):  # method belogs to abbreviation extraction
    upper_cases = ''.join([c for c in w if c.isupper()])
    return len(upper_cases) / len(w)


def check_if_word_is_abbreviation(word):  # method belogs to abbreviation extraction
    return (len(word.split()) == 1 and len(word) <= 13 and portion_of_capital_letters(word) >= 0.29)


def clean_string_from_puntuation(s):
    invalidcharacters = set(string.punctuation)
    if any(char in invalidcharacters for char in s):
        s_ = s.translate(str.maketrans('', '', string.punctuation))
    else:
        s_ = s
    return s_


def remove_determiners(nc, language_flag):
    nc_splitted = nc.split()
    if language_flag == "EN":
        irrelevant_determiners = ["each", "the", "an", "each", "them", "therefore", "e.g.", ",", "a", "one", "on", "at",
                                  "by", "when", "such", "all", "this", "given", "its", "their"]
    else:
        irrelevant_determiners = ["der", "die", "das", "dem", "den", "ein", "eine", "des", "bei", "beim"]

    term_changed = False
    while nc_splitted[0].lower() in irrelevant_determiners:
        cleaned_nc = ' '.join([w for w in nc_splitted[1:]])
        nc_splitted = nc_splitted[1:]
        term_changed = True
        
        if len(nc_splitted) == 0:
            return None

    if not term_changed:
        cleaned_nc = nc

    return cleaned_nc.strip()


def advanced_lemmatizer(l_tok, language_flag):
    invalidcharacters = set(string.punctuation)
    if any(char in invalidcharacters for char in l_tok):
        return l_tok
    if check_if_word_is_abbreviation(l_tok):
        return l_tok
    if language_flag == "EN":
        new_doc = nlp_en(l_tok)
    else:
        new_doc = nlp_de(l_tok)
    return new_doc[-1].lemma_


def normalize_nc(nc, language_flag):
    if len(nc.split()) <= 1:
        return advanced_lemmatizer(nc, language_flag)

    cleaned_nc = re.sub(r"[\([{})\]]", "", nc)
    cleaned_nc = re.sub("(\w)([,|.])(\s)", r"\1 ", cleaned_nc)
    splitted_cleaned_nc = cleaned_nc.split()
    if len(splitted_cleaned_nc) <= 1:
        return advanced_lemmatizer(nc, language_flag)

    last_token = splitted_cleaned_nc[-1]
    splitted_cleaned_nc.pop()
    splitted_cleaned_nc.append(advanced_lemmatizer(last_token, language_flag))
    return " ".join(tok for tok in splitted_cleaned_nc)

## Stopword list sources


1. https://gist.github.com/sebleier/554280
2. https://countwordsfree.com/stopwords (english)
3. https://countwordsfree.com/stopwords (deutsch)
4. https://solariz.de/de/downloads/6/german-enhanced-stopwords.htm

In [5]:
stopwords_en_1 = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz",]
stopwords_en_2 = ["able", "about", "above", "abroad", "according", "accordingly", "across", "actually", "adj", "after", "afterwards", "again", "against", "ago", "ahead", "ain't", "all", "allow", "allows", "almost", "alone", "along", "alongside", "already", "also", "although", "always", "am", "amid", "amidst", "among", "amongst", "an", "and", "another", "any", "anybody", "anyhow", "anyone", "anything", "anyway", "anyways", "anywhere", "apart", "appear", "appreciate", "appropriate", "are", "aren't", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "available", "away", "awfully", "back", "backward", "backwards", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "both", "brief", "but", "by", "came", "can", "cannot", "cant", "can't", "caption", "cause", "causes", "certain", "certainly", "changes", "clearly", "c'mon", "co", "co.", "com", "come", "comes", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn't", "course", "c's", "currently", "dare", "daren't", "definitely", "described", "despite", "did", "didn't", "different", "directly", "do", "does", "doesn't", "doing", "done", "don't", "down", "downwards", "during", "each", "edu", "eg", "eight", "eighty", "either", "else", "elsewhere", "end", "ending", "enough", "entirely", "especially", "et", "etc", "even", "ever", "evermore", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "fairly", "far", "farther", "few", "fewer", "fifth", "first", "five", "followed", "following", "follows", "for", "forever", "former", "formerly", "forth", "forward", "found", "four", "from", "further", "furthermore", "get", "gets", "getting", "given", "gives", "go", "goes", "going", "gone", "got", "gotten", "greetings", "had", "hadn't", "half", "happens", "hardly", "has", "hasn't", "have", "haven't", "having", "he", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "here's", "hereupon", "hers", "herself", "he's", "hi", "him", "himself", "his", "hither", "hopefully", "how", "howbeit", "however", "hundred", "i'd", "ie", "if", "ignored", "i'll", "i'm", "immediate", "in", "inasmuch", "inc", "inc.", "indeed", "indicate", "indicated", "indicates", "inner", "inside", "insofar", "instead", "into", "inward", "is", "isn't", "it", "it'd", "it'll", "its", "it's", "itself", "i've", "just", "k", "keep", "keeps", "kept", "know", "known", "knows", "last", "lately", "later", "latter", "latterly", "least", "less", "lest", "let", "let's", "like", "liked", "likely", "likewise", "little", "look", "looking", "looks", "low", "lower", "ltd", "made", "mainly", "make", "makes", "many", "may", "maybe", "mayn't", "me", "mean", "meantime", "meanwhile", "merely", "might", "mightn't", "mine", "minus", "miss", "more", "moreover", "most", "mostly", "mr", "mrs", "much", "must", "mustn't", "my", "myself", "name", "namely", "nd", "near", "nearly", "necessary", "need", "needn't", "needs", "neither", "never", "neverf", "neverless", "nevertheless", "new", "next", "nine", "ninety", "no", "nobody", "non", "none", "nonetheless", "noone", "no-one", "nor", "normally", "not", "nothing", "notwithstanding", "novel", "now", "nowhere", "obviously", "of", "off", "often", "oh", "ok", "okay", "old", "on", "once", "one", "ones", "one's", "only", "onto", "opposite", "or", "other", "others", "otherwise", "ought", "oughtn't", "our", "ours", "ourselves", "out", "outside", "over", "overall", "own", "particular", "particularly", "past", "per", "perhaps", "placed", "please", "plus", "possible", "presumably", "probably", "provided", "provides", "que", "quite", "qv", "rather", "rd", "re", "really", "reasonably", "recent", "recently", "regarding", "regardless", "regards", "relatively", "respectively", "right", "round", "said", "same", "saw", "say", "saying", "says", "second", "secondly", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "shall", "shan't", "she", "she'd", "she'll", "she's", "should", "shouldn't", "since", "six", "so", "some", "somebody", "someday", "somehow", "someone", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "specified", "specify", "specifying", "still", "sub", "such", "sup", "sure", "take", "taken", "taking", "tell", "tends", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "there'd", "therefore", "therein", "there'll", "there're", "theres", "there's", "thereupon", "there've", "these", "they", "they'd", "they'll", "they're", "they've", "thing", "things", "think", "third", "thirty", "this", "thorough", "thoroughly", "those", "though", "three", "through", "throughout", "thru", "thus", "till", "to", "together", "too", "took", "toward", "towards", "tried", "tries", "truly", "try", "trying", "t's", "twice", "two", "un", "under", "underneath", "undoing", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "up", "upon", "upwards", "us", "use", "used", "useful", "uses", "using", "usually", "v", "value", "various", "versus", "very", "via", "viz", "vs", "want", "wants", "was", "wasn't", "way", "we", "we'd", "welcome", "well", "we'll", "went", "were", "we're", "weren't", "we've", "what", "whatever", "what'll", "what's", "what've", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "where's", "whereupon", "wherever", "whether", "which", "whichever", "while", "whilst", "whither", "who", "who'd", "whoever", "whole", "who'll", "whom", "whomever", "who's", "whose", "why", "will", "willing", "wish", "with", "within", "without", "wonder", "won't", "would", "wouldn't", "yes", "yet", "you", "you'd", "you'll", "your", "you're", "yours", "yourself", "yourselves", "you've", "zero", "a", "how's", "i", "when's", "why's", "b", "c", "d", "e", "f", "g", "h", "j", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "uucp", "w", "x", "y", "z", "I", "www", "amount", "bill", "bottom", "call", "computer", "con", "couldnt", "cry", "de", "describe", "detail", "due", "eleven", "empty", "fifteen", "fifty", "fill", "find", "fire", "forty", "front", "full", "give", "hasnt", "herse", "himse", "interest", "itse”", "mill", "move", "myse”", "part", "put", "show", "side", "sincere", "sixty", "system", "ten", "thick", "thin", "top", "twelve", "twenty", "abst", "accordance", "act", "added", "adopted", "affected", "affecting", "affects", "ah", "announce", "anymore", "apparently", "approximately", "aren", "arent", "arise", "auth", "beginning", "beginnings", "begins", "biol", "briefly", "ca", "date", "ed", "effect", "et-al", "ff", "fix", "gave", "giving", "heres", "hes", "hid", "home", "id", "im", "immediately", "importance", "important", "index", "information", "invention", "itd", "keys", "kg", "km", "largely", "lets", "line", "'ll", "means", "mg", "million", "ml", "mug", "na", "nay", "necessarily", "nos", "noted", "obtain", "obtained", "omitted", "ord", "owing", "page", "pages", "poorly", "possibly", "potentially", "pp", "predominantly", "present", "previously", "primarily", "promptly", "proud", "quickly", "ran", "readily", "ref", "refs", "related", "research", "resulted", "resulting", "results", "run", "sec", "section", "shed", "shes", "showed", "shown", "showns", "shows", "significant", "significantly", "similar", "similarly", "slightly", "somethan", "specifically", "state", "states", "stop", "strongly", "substantially", "successfully", "sufficiently", "suggest", "thered", "thereof", "therere", "thereto", "theyd", "theyre", "thou", "thoughh", "thousand", "throug", "til", "tip", "ts", "ups", "usefully", "usefulness", "'ve", "vol", "vols", "wed", "whats", "wheres", "whim", "whod", "whos", "widely", "words", "world", "youd", "youre"]

stopwords_de_1 = ["a", "ab", "aber", "ach", "acht", "achte", "achten", "achter", "achtes", "ag", "alle", "allein", "allem", "allen", "aller", "allerdings", "alles", "allgemeinen", "als", "also", "am", "an", "andere", "anderen", "andern", "anders", "au", "auch", "auf", "aus", "ausser", "außer", "ausserdem", "außerdem", "b", "bald", "bei", "beide", "beiden", "beim", "beispiel", "bekannt", "bereits", "besonders", "besser", "besten", "bin", "bis", "bisher", "bist", "c", "d", "da", "dabei", "dadurch", "dafür", "dagegen", "daher", "dahin", "dahinter", "damals", "damit", "danach", "daneben", "dank", "dann", "daran", "darauf", "daraus", "darf", "darfst", "darin", "darüber", "darum", "darunter", "das", "dasein", "daselbst", "dass", "daß", "dasselbe", "davon", "davor", "dazu", "dazwischen", "dein", "deine", "deinem", "deiner", "dem", "dementsprechend", "demgegenüber", "demgemäss", "demgemäß", "demselben", "demzufolge", "den", "denen", "denn", "denselben", "der", "deren", "derjenige", "derjenigen", "dermassen", "dermaßen", "derselbe", "derselben", "des", "deshalb", "desselben", "dessen", "deswegen", "d.h", "dich", "die", "diejenige", "diejenigen", "dies", "diese", "dieselbe", "dieselben", "diesem", "diesen", "dieser", "dieses", "dir", "doch", "dort", "drei", "drin", "dritte", "dritten", "dritter", "drittes", "du", "durch", "durchaus", "dürfen", "dürft", "durfte", "durften", "e", "eben", "ebenso", "ehrlich", "ei", "ei,", "eigen", "eigene", "eigenen", "eigener", "eigenes", "ein", "einander", "eine", "einem", "einen", "einer", "eines", "einige", "einigen", "einiger", "einiges", "einmal", "eins", "elf", "en", "ende", "endlich", "entweder", "er", "Ernst", "erst", "erste", "ersten", "erster", "erstes", "es", "etwa", "etwas", "euch", "f", "früher", "fünf", "fünfte", "fünften", "fünfter", "fünftes", "für", "g", "gab", "ganz", "ganze", "ganzen", "ganzer", "ganzes", "gar", "gedurft", "gegen", "gegenüber", "gehabt", "gehen", "geht", "gekannt", "gekonnt", "gemacht", "gemocht", "gemusst", "genug", "gerade", "gern", "gesagt", "geschweige", "gewesen", "gewollt", "geworden", "gibt", "ging", "gleich", "gott", "gross", "groß", "grosse", "große", "grossen", "großen", "grosser", "großer", "grosses", "großes", "gut", "gute", "guter", "gutes", "h", "habe", "haben", "habt", "hast", "hat", "hatte", "hätte", "hatten", "hätten", "heisst", "her", "heute", "hier", "hin", "hinter", "hoch", "i", "ich", "ihm", "ihn", "ihnen", "ihr", "ihre", "ihrem", "ihren", "ihrer", "ihres", "im", "immer", "in", "indem", "infolgedessen", "ins", "irgend", "ist", "j", "ja", "jahr", "jahre", "jahren", "je", "jede", "jedem", "jeden", "jeder", "jedermann", "jedermanns", "jedoch", "jemand", "jemandem", "jemanden", "jene", "jenem", "jenen", "jener", "jenes", "jetzt", "k", "kam", "kann", "kannst", "kaum", "kein", "keine", "keinem", "keinen", "keiner", "kleine", "kleinen", "kleiner", "kleines", "kommen", "kommt", "können", "könnt", "konnte", "könnte", "konnten", "kurz", "l", "lang", "lange", "leicht", "leide", "lieber", "los", "m", "machen", "macht", "machte", "mag", "magst", "mahn", "man", "manche", "manchem", "manchen", "mancher", "manches", "mann", "mehr", "mein", "meine", "meinem", "meinen", "meiner", "meines", "mensch", "menschen", "mich", "mir", "mit", "mittel", "mochte", "möchte", "mochten", "mögen", "möglich", "mögt", "morgen", "muss", "muß", "müssen", "musst", "müsst", "musste", "mussten", "n", "na", "nach", "nachdem", "nahm", "natürlich", "neben", "nein", "neue", "neuen", "neun", "neunte", "neunten", "neunter", "neuntes", "nicht", "nichts", "nie", "niemand", "niemandem", "niemanden", "noch", "nun", "nur", "o", "ob", "oben", "oder", "offen", "oft", "ohne", "Ordnung", "p", "q", "r", "recht", "rechte", "rechten", "rechter", "rechtes", "richtig", "rund", "s", "sa", "sache", "sagt", "sagte", "sah", "satt", "schlecht", "Schluss", "schon", "sechs", "sechste", "sechsten", "sechster", "sechstes", "sehr", "sei", "seid", "seien", "sein", "seine", "seinem", "seinen", "seiner", "seines", "seit", "seitdem", "selbst", "sich", "sie", "sieben", "siebente", "siebenten", "siebenter", "siebentes", "sind", "so", "solang", "solche", "solchem", "solchen", "solcher", "solches", "soll", "sollen", "sollte", "sollten", "sondern", "sonst", "sowie", "später", "statt", "t", "tag", "tage", "tagen", "tat", "teil", "tel", "tritt", "trotzdem", "tun", "u", "über", "überhaupt", "übrigens", "uhr", "um", "und", "und?", "uns", "unser", "unsere", "unserer", "unter", "v", "vergangenen", "viel", "viele", "vielem", "vielen", "vielleicht", "vier", "vierte", "vierten", "vierter", "viertes", "vom", "von", "vor", "w", "wahr?", "während", "währenddem", "währenddessen", "wann", "war", "wäre", "waren", "wart", "warum", "was", "wegen", "weil", "weit", "weiter", "weitere", "weiteren", "weiteres", "welche", "welchem", "welchen", "welcher", "welches", "wem", "wen", "wenig", "wenige", "weniger", "weniges", "wenigstens", "wenn", "wer", "werde", "werden", "werdet", "wessen", "wie", "wieder", "will", "willst", "wir", "wird", "wirklich", "wirst", "wo", "wohl", "wollen", "wollt", "wollte", "wollten", "worden", "wurde", "würde", "wurden", "würden", "x", "y", "z", "z.b", "zehn", "zehnte", "zehnten", "zehnter", "zehntes", "zeit", "zu", "zuerst", "zugleich", "zum", "zunächst", "zur", "zurück", "zusammen", "zwanzig", "zwar", "zwei", "zweite", "zweiten", "zweiter", "zweites", "zwischen", "zwölf", "euer", "eure", "hattest", "hattet", "jedes", "mußt", "müßt", "sollst", "sollt", "soweit", "weshalb", "wieso", "woher", "wohin"]
stopwords_de_2 = ["ab", "aber", "abermaliges", "abermals", "abgerufen", "abgerufene", "abgerufener", "abgerufenes", "abgesehen", "acht", "aehnlich", "aehnliche", "aehnlichem", "aehnlichen", "aehnlicher", "aehnliches", "aehnlichste", "aehnlichstem", "aehnlichsten", "aehnlichster", "aehnlichstes", "aeusserst", "aeusserste", "aeusserstem", "aeussersten", "aeusserster", "aeusserstes", "ähnlich", "ähnliche", "ähnlichem", "ähnlichen", "ähnlicher", "ähnliches", "ähnlichst", "ähnlichste", "ähnlichstem", "ähnlichsten", "ähnlichster", "ähnlichstes", "alle", "allein", "alleine", "allem", "allemal", "allen", "allenfalls", "allenthalben", "aller", "allerdings", "allerlei", "alles", "allesamt", "allg", "allg.", "allgemein", "allgemeine", "allgemeinem", "allgemeinen", "allgemeiner", "allgemeines", "allgemeinste", "allgemeinstem", "allgemeinsten", "allgemeinster", "allgemeinstes", "allmählich", "allzeit", "allzu", "als", "alsbald", "also", "am", "an", "and", "andauernd", "andauernde", "andauerndem", "andauernden", "andauernder", "andauerndes", "ander", "andere", "anderem", "anderen", "anderenfalls", "anderer", "andererseits", "anderes", "anderm", "andern", "andernfalls", "anderr", "anders", "anderst", "anderweitig", "anderweitige", "anderweitigem", "anderweitigen", "anderweitiger", "anderweitiges", "anerkannt", "anerkannte", "anerkannter", "anerkanntes", "anfangen", "anfing", "angefangen", "angesetze", "angesetzt", "angesetzten", "angesetzter", "ans", "anscheinend", "ansetzen", "ansonst", "ansonsten", "anstatt", "anstelle", "arbeiten", "auch", "auf", "aufgehört", "aufgrund", "aufhören", "aufhörte", "aufzusuchen", "augenscheinlich", "augenscheinliche", "augenscheinlichem", "augenscheinlichen", "augenscheinlicher", "augenscheinliches", "augenscheinlichst", "augenscheinlichste", "augenscheinlichstem", "augenscheinlichsten", "augenscheinlichster", "augenscheinlichstes", "aus", "ausdrücken", "ausdrücklich", "ausdrückliche", "ausdrücklichem", "ausdrücklichen", "ausdrücklicher", "ausdrückliches", "ausdrückt", "ausdrückte", "ausgenommen", "ausgenommene", "ausgenommenem", "ausgenommenen", "ausgenommener", "ausgenommenes", "ausgerechnet", "ausgerechnete", "ausgerechnetem", "ausgerechneten", "ausgerechneter", "ausgerechnetes", "ausnahmslos", "ausnahmslose", "ausnahmslosem", "ausnahmslosen", "ausnahmsloser", "ausnahmsloses", "außen", "ausser", "ausserdem", "außerhalb", "äusserst", "äusserste", "äusserstem", "äussersten", "äusserster", "äusserstes", "author", "autor", "baelde", "bald", "bälde", "bearbeite", "bearbeiten", "bearbeitete", "bearbeiteten", "bedarf", "bedürfen", "bedurfte", "been", "befahl", "befiehlt", "befiehlte", "befohlene", "befohlens", "befragen", "befragte", "befragten", "befragter", "begann", "beginnen", "begonnen", "behalten", "behielt", "bei", "beide", "beidem", "beiden", "beider", "beiderlei", "beides", "beim", "beinahe", "beisammen", "beispielsweise", "beitragen", "beitrugen", "bekannt", "bekannte", "bekannter", "bekanntlich", "bekanntliche", "bekanntlichem", "bekanntlichen", "bekanntlicher", "bekanntliches", "bekennen", "benutzt", "bereits", "berichten", "berichtet", "berichtete", "berichteten", "besonders", "besser", "bessere", "besserem", "besseren", "besserer", "besseres", "bestehen", "besteht", "bestenfalls", "bestimmt", "bestimmte", "bestimmtem", "bestimmten", "bestimmter", "bestimmtes", "beträchtlich", "beträchtliche", "beträchtlichem", "beträchtlichen", "beträchtlicher", "beträchtliches", "betraechtlich", "betraechtliche", "betraechtlichem", "betraechtlichen", "betraechtlicher", "betraechtliches", "betreffend", "betreffende", "betreffendem", "betreffenden", "betreffender", "betreffendes", "bevor", "bez", "bez.", "bezgl", "bezgl.", "bezueglich", "bezüglich", "bietet", "bin", "bis", "bisher", "bisherige", "bisherigem", "bisherigen", "bisheriger", "bisheriges", "bislang", "bisschen", "bist", "bitte", "bleiben", "bleibt", "blieb", "bloss", "böden", "boeden", "brachte", "brachten", "brauchen", "braucht", "bräuchte", "bringen", "bsp", "bsp.", "bspw", "bspw.", "bzw", "bzw.", "ca", "ca.", "circa", "da", "dabei", "dadurch", "dafuer", "dafür", "dagegen", "daher", "dahin", "dahingehend", "dahingehende", "dahingehendem", "dahingehenden", "dahingehender", "dahingehendes", "dahinter", "damalige", "damaligem", "damaligen", "damaliger", "damaliges", "damals", "damit", "danach", "daneben", "dank", "danke", "danken", "dann", "dannen", "daran", "darauf", "daraus", "darf", "darfst", "darin", "darüber", "darüberhinaus", "darueber", "darueberhinaus", "darum", "darunter", "das", "daß", "dass", "dasselbe", "Dat", "davon", "davor", "dazu", "dazwischen", "dein", "deine", "deinem", "deinen", "deiner", "deines", "dem", "demgegenüber", "demgegenueber", "demgemaess", "demgemäss", "demnach", "demselben", "den", "denen", "denkbar", "denkbare", "denkbarem", "denkbaren", "denkbarer", "denkbares", "denn", "dennoch", "denselben", "der", "derart", "derartig", "derartige", "derartigem", "derartigen", "derartiger", "derem", "deren", "derer", "derjenige", "derjenigen", "derselbe", "derselben", "derzeit", "derzeitig", "derzeitige", "derzeitigem", "derzeitigen", "derzeitiges", "des", "deshalb", "desselben", "dessen", "dessenungeachtet", "desto", "desungeachtet", "deswegen", "dich", "die", "diejenige", "diejenigen", "dies", "diese", "dieselbe", "dieselben", "diesem", "diesen", "dieser", "dieses", "diesseitig", "diesseitige", "diesseitigem", "diesseitigen", "diesseitiger", "diesseitiges", "diesseits", "dinge", "dir", "direkt", "direkte", "direkten", "direkter", "doch", "doppelt", "dort", "dorther", "dorthin", "dran", "drauf", "drei", "dreißig", "drin", "dritte", "drüber", "drueber", "drum", "drunter", "du", "duerfte", "duerften", "duerftest", "duerftet", "dunklen", "durch", "durchaus", "durchweg", "durchwegs", "dürfen", "durfte", "dürfte", "durften", "dürften", "durftest", "dürftest", "durftet", "dürftet", "eben", "ebenfalls", "ebenso", "ect", "ect.", "ehe", "eher", "eheste", "ehestem", "ehesten", "ehester", "ehestes", "eigen", "eigene", "eigenem", "eigenen", "eigener", "eigenes", "eigenst", "eigentlich", "eigentliche", "eigentlichem", "eigentlichen", "eigentlicher", "eigentliches", "ein", "einbaün", "eine", "einem", "einen", "einer", "einerlei", "einerseits", "eines", "einfach", "einführen", "einführte", "einführten", "eingesetzt", "einig", "einige", "einigem", "einigen", "einiger", "einigermaßen", "einiges", "einmal", "einmalig", "einmalige", "einmaligem", "einmaligen", "einmaliger", "einmaliges", "eins", "einseitig", "einseitige", "einseitigen", "einseitiger", "einst", "einstmals", "einzig", "empfunden", "ende", "entgegen", "entlang", "entsprechend", "entsprechende", "entsprechendem", "entsprechenden", "entsprechender", "entsprechendes", "entweder", "er", "ergänze", "ergänzen", "ergänzte", "ergänzten", "ergo", "erhält", "erhalten", "erhielt", "erhielten", "erneut", "eröffne", "eröffnen", "eröffnet", "eröffnete", "eröffnetes", "erscheinen", "erst", "erste", "erstem", "ersten", "erster", "erstere", "ersterem", "ersteren", "ersterer", "ersteres", "erstes", "es", "etc", "etc.", "etliche", "etlichem", "etlichen", "etlicher", "etliches", "etwa", "etwaige", "etwas", "euch", "euer", "eure", "eurem", "euren", "eurer", "eures", "euretwegen", "fall", "falls", "fand", "fast", "ferner", "fertig", "finde", "finden", "findest", "findet", "folgend", "folgende", "folgendem", "folgenden", "folgender", "folgendermassen", "folgendes", "folglich", "for", "fordern", "fordert", "forderte", "forderten", "fort", "fortsetzen", "fortsetzt", "fortsetzte", "fortsetzten", "fragte", "frau", "frei", "freie", "freier", "freies", "fuer", "fuers", "fünf", "für", "fürs", "gab", "gaenzlich", "gaenzliche", "gaenzlichem", "gaenzlichen", "gaenzlicher", "gaenzliches", "gängig", "gängige", "gängigen", "gängiger", "gängiges", "ganz", "ganze", "ganzem", "ganzen", "ganzer", "ganzes", "gänzlich", "gänzliche", "gänzlichem", "gänzlichen", "gänzlicher", "gänzliches", "gar", "gbr", "geb", "geben", "geblieben", "gebracht", "gedurft", "geehrt", "geehrte", "geehrten", "geehrter", "gefallen", "gefälligst", "gefällt", "gefiel", "gegeben", "gegen", "gegenüber", "gegenueber", "gehabt", "gehalten", "gehen", "geht", "gekommen", "gekonnt", "gemacht", "gemaess", "gemäss", "gemeinhin", "gemocht", "genau", "genommen", "genug", "gepriesener", "gepriesenes", "gerade", "gern", "gesagt", "gesehen", "gestern", "gestrige", "getan", "geteilt", "geteilte", "getragen", "getrennt", "gewesen", "gewiss", "gewisse", "gewissem", "gewissen", "gewisser", "gewissermaßen", "gewisses", "gewollt", "geworden", "ggf", "ggf.", "gib", "gibt", "gilt", "gleich", "gleiche", "gleichem", "gleichen", "gleicher", "gleiches", "gleichsam", "gleichste", "gleichstem", "gleichsten", "gleichster", "gleichstes", "gleichwohl", "gleichzeitig", "gleichzeitige", "gleichzeitigem", "gleichzeitigen", "gleichzeitiger", "gleichzeitiges", "glücklicherweise", "gluecklicherweise", "gmbh", "gottseidank", "gratulieren", "gratuliert", "gratulierte", "groesstenteils", "grösstenteils", "gruendlich", "gründlich", "gut", "gute", "guten", "hab", "habe", "haben", "habt", "haette", "haeufig", "haeufige", "haeufigem", "haeufigen", "haeufiger", "haeufigere", "haeufigeren", "haeufigerer", "haeufigeres", "halb", "hallo", "halten", "hast", "hat", "hätt", "hatte", "hätte", "hatten", "hätten", "hattest", "hattet", "häufig", "häufige", "häufigem", "häufigen", "häufiger", "häufigere", "häufigeren", "häufigerer", "häufigeres", "hen", "her", "heraus", "herein", "herum", "heute", "heutige", "heutigem", "heutigen", "heutiger", "heutiges", "hier", "hierbei", "hiermit", "hiesige", "hiesigem", "hiesigen", "hiesiger", "hiesiges", "hin", "hindurch", "hinein", "hingegen", "hinlanglich", "hinlänglich", "hinten", "hintendran", "hinter", "hinterher", "hinterm", "hintern", "hinunter", "hoch", "höchst", "höchstens", "http", "hundert", "ich", "igitt", "ihm", "ihn", "ihnen", "ihr", "ihre", "ihrem", "ihren", "ihrer", "ihres", "ihretwegen", "ihrige", "ihrigen", "ihriges", "im", "immer", "immerhin", "immerwaehrend", "immerwaehrende", "immerwaehrendem", "immerwaehrenden", "immerwaehrender", "immerwaehrendes", "immerwährend", "immerwährende", "immerwährendem", "immerwährenden", "immerwährender", "immerwährendes", "immerzu", "important", "in", "indem", "indessen", "Inf.", "info", "infolge", "infolgedessen", "information", "innen", "innerhalb", "innerlich", "ins", "insbesondere", "insgeheim", "insgeheime", "insgeheimer", "insgesamt", "insgesamte", "insgesamter", "insofern", "inzwischen", "irgend", "irgendein", "irgendeine", "irgendeinem", "irgendeiner", "irgendeines", "irgendetwas", "irgendjemand", "irgendjemandem", "irgendwann", "irgendwas", "irgendwelche", "irgendwen", "irgendwenn", "irgendwer", "irgendwie", "irgendwo", "irgendwohin", "ist", "ja", "jaehrig", "jaehrige", "jaehrigem", "jaehrigen", "jaehriger", "jaehriges", "jährig", "jährige", "jährigem", "jährigen", "jähriges", "je", "jede", "jedem", "jeden", "jedenfalls", "jeder", "jederlei", "jedes", "jedesmal", "jedoch", "jeglichem", "jeglichen", "jeglicher", "jegliches", "jemals", "jemand", "jemandem", "jemanden", "jemandes", "jene", "jenem", "jenen", "jener", "jenes", "jenseitig", "jenseitigem", "jenseitiger", "jenseits", "jetzt", "jung", "junge", "jungem", "jungen", "junger", "junges", "kaeumlich", "kam", "kann", "kannst", "kaum", "käumlich", "kein", "keine", "keinem", "keinen", "keiner", "keinerlei", "keines", "keineswegs", "klar", "klare", "klaren", "klares", "klein", "kleinen", "kleiner", "kleines", "koennen", "koennt", "koennte", "koennten", "koenntest", "koenntet", "komme", "kommen", "kommt", "konkret", "konkrete", "konkreten", "konkreter", "konkretes", "könn", "können", "könnt", "konnte", "könnte", "konnten", "könnten", "konntest", "könntest", "konntet", "könntet", "kuenftig", "kuerzlich", "kuerzlichst", "künftig", "kürzlich", "kürzlichst", "laengst", "lag", "lagen", "langsam", "längst", "längstens", "lassen", "laut", "lediglich", "leer", "legen", "legte", "legten", "leicht", "leider", "lesen", "letze", "letzte", "letzten", "letztendlich", "letztens", "letztere", "letzterem", "letzterer", "letzteres", "letztes", "letztlich", "lichten", "liegt", "liest", "links", "mache", "machen", "machst", "macht", "machte", "machten", "mag", "magst", "mal", "man", "manch", "manche", "manchem", "manchen", "mancher", "mancherlei", "mancherorts", "manches", "manchmal", "mann", "margin", "massgebend", "massgebende", "massgebendem", "massgebenden", "massgebender", "massgebendes", "massgeblich", "massgebliche", "massgeblichem", "massgeblichen", "massgeblicher", "mehr", "mehrere", "mehrerer", "mehrfach", "mehrmalig", "mehrmaligem", "mehrmaliger", "mehrmaliges", "mein", "meine", "meinem", "meinen", "meiner", "meines", "meinetwegen", "meins", "meist", "meiste", "meisten", "meistens", "meistenteils", "meta", "mich", "mindestens", "mir", "mit", "miteinander", "mitgleich", "mithin", "mitnichten", "mittels", "mittelst", "mitten", "mittig", "mitunter", "mitwohl", "mochte", "möchte", "möchten", "möchtest", "moechte", "moeglich", "moeglichst", "moeglichste", "moeglichstem", "moeglichsten", "moeglichster", "mögen", "möglich", "mögliche", "möglichen", "möglicher", "möglicherweise", "möglichst", "möglichste", "möglichstem", "möglichsten", "möglichster", "morgen", "morgige", "muessen", "muesst", "muesste", "muss", "müssen", "musst", "müßt", "musste", "müsste", "mussten", "müssten", "nach", "nachdem", "nacher", "nachher", "nachhinein", "nächste", "nacht", "naechste", "naemlich", "nahm", "nämlich", "naturgemaess", "naturgemäss", "natürlich", "ncht", "neben", "nebenan", "nehmen", "nein", "neu", "neue", "neuem", "neuen", "neuer", "neuerdings", "neuerlich", "neuerliche", "neuerlichem", "neuerlicher", "neuerliches", "neues", "neulich", "neun", "nicht", "nichts", "nichtsdestotrotz", "nichtsdestoweniger", "nie", "niemals", "niemand", "niemandem", "niemanden", "niemandes", "nimm", "nimmer", "nimmt", "nirgends", "nirgendwo", "noch", "noetigenfalls", "nötigenfalls", "nun", "nur", "nutzen", "nutzt", "nützt", "nutzung", "ob", "oben", "ober", "oberen", "oberer", "oberhalb", "oberste", "obersten", "oberster", "obgleich", "obs", "obschon", "obwohl", "oder", "oefter", "oefters", "off", "offenkundig", "offenkundige", "offenkundigem", "offenkundigen", "offenkundiger", "offenkundiges", "offensichtlich", "offensichtliche", "offensichtlichem", "offensichtlichen", "offensichtlicher", "offensichtliches", "oft", "öfter", "öfters", "oftmals", "ohne", "ohnedies", "online", "paar", "partout", "per", "persoenlich", "persoenliche", "persoenlichem", "persoenlicher", "persoenliches", "persönlich", "persönliche", "persönlicher", "persönliches", "pfui", "ploetzlich", "ploetzliche", "ploetzlichem", "ploetzlicher", "ploetzliches", "plötzlich", "plötzliche", "plötzlichem", "plötzlicher", "plötzliches", "pro", "quasi", "reagiere", "reagieren", "reagiert", "reagierte", "recht", "rechts", "regelmäßig", "reichlich", "reichliche", "reichlichem", "reichlichen", "reichlicher", "restlos", "restlose", "restlosem", "restlosen", "restloser", "restloses", "richtiggehend", "richtiggehende", "richtiggehendem", "richtiggehenden", "richtiggehender", "richtiggehendes", "rief", "rund", "rundheraus", "rundum", "runter", "sage", "sagen", "sagt", "sagte", "sagten", "sagtest", "sagtet", "samt", "sämtliche", "sang", "sangen", "sattsam", "schätzen", "schätzt", "schätzte", "schätzten", "scheinbar", "scheinen", "schlechter", "schlicht", "schlichtweg", "schließlich", "schlussendlich", "schnell", "schon", "schreibe", "schreiben", "schreibens", "schreiber", "schwerlich", "schwerliche", "schwerlichem", "schwerlichen", "schwerlicher", "schwerliches", "schwierig", "sechs", "sect", "sehe", "sehen", "sehr", "sehrwohl", "seht", "sei", "seid", "seien", "seiest", "seiet", "sein", "seine", "seinem", "seinen", "seiner", "seines", "seit", "seitdem", "seite", "seiten", "seither", "selbe", "selben", "selber", "selbst", "selbstredend", "selbstredende", "selbstredendem", "selbstredenden", "selbstredender", "selbstredendes", "seltsamerweise", "senke", "senken", "senkt", "senkte", "senkten", "setzen", "setzt", "setzte", "setzten", "sich", "sicher", "sicherlich", "sie", "sieben", "siebte", "siehe", "sieht", "sind", "singen", "singt", "so", "sobald", "sodaß", "soeben", "sofern", "sofort", "sog", "sogar", "sogleich", "solange", "solc", "solc hen", "solch", "solche", "solchem", "solchen", "solcher", "solches", "soll", "sollen", "sollst", "sollt", "sollte", "sollten", "solltest", "solltet", "somit", "sondern", "sonst", "sonstig", "sonstige", "sonstigem", "sonstiger", "sonstwo", "sooft", "soviel", "soweit", "sowie", "sowieso", "sowohl", "später", "spielen", "startet", "startete", "starteten", "statt", "stattdessen", "steht", "steige", "steigen", "steigt", "stellenweise", "stellenweisem", "stellenweisen", "stets", "stieg", "stiegen", "such", "suchen", "tages", "tat", "tät", "tatsächlich", "tatsächlichen", "tatsächlicher", "tatsächliches", "tatsaechlich", "tatsaechlichen", "tatsaechlicher", "tatsaechliches", "tausend", "teile", "teilen", "teilte", "teilten", "tief", "titel", "toll", "total", "trage", "tragen", "trägt", "trotzdem", "trug", "tun", "tust", "tut", "txt", "übel", "über", "überall", "überallhin", "überaus", "überdies", "überhaupt", "überll", "übermorgen", "üblicherweise", "übrig", "übrigens", "ueber", "ueberall", "ueberallhin", "ueberaus", "ueberdies", "ueberhaupt", "uebermorgen", "ueblicherweise", "uebrig", "uebrigens", "um", "ums", "umso", "umstaendehalber", "umständehalber", "unbedingt", "unbedingte", "unbedingter", "unbedingtes", "und", "unerhoert", "unerhoerte", "unerhoertem", "unerhoerten", "unerhoerter", "unerhoertes", "unerhört", "unerhörte", "unerhörtem", "unerhörten", "unerhörter", "unerhörtes", "ungefähr", "ungemein", "ungewoehnlich", "ungewoehnliche", "ungewoehnlichem", "ungewoehnlichen", "ungewoehnlicher", "ungewoehnliches", "ungewöhnlich", "ungewöhnliche", "ungewöhnlichem", "ungewöhnlichen", "ungewöhnlicher", "ungewöhnliches", "ungleich", "ungleiche", "ungleichem", "ungleichen", "ungleicher", "ungleiches", "unmassgeblich", "unmassgebliche", "unmassgeblichem", "unmassgeblichen", "unmassgeblicher", "unmassgebliches", "unmoeglich", "unmoegliche", "unmoeglichem", "unmoeglichen", "unmoeglicher", "unmoegliches", "unmöglich", "unmögliche", "unmöglichen", "unmöglicher", "unnötig", "uns", "unsaeglich", "unsaegliche", "unsaeglichem", "unsaeglichen", "unsaeglicher", "unsaegliches", "unsagbar", "unsagbare", "unsagbarem", "unsagbaren", "unsagbarer", "unsagbares", "unsäglich", "unsägliche", "unsäglichem", "unsäglichen", "unsäglicher", "unsägliches", "unse", "unsem", "unsen", "unser", "unsere", "unserem", "unseren", "unserer", "unseres", "unserm", "unses", "unsre", "unsrem", "unsren", "unsrer", "unsres", "unstreitig", "unstreitige", "unstreitigem", "unstreitigen", "unstreitiger", "unstreitiges", "unten", "unter", "unterbrach", "unterbrechen", "untere", "unterem", "unteres", "unterhalb", "unterste", "unterster", "unterstes", "unwichtig", "unzweifelhaft", "unzweifelhafte", "unzweifelhaftem", "unzweifelhaften", "unzweifelhafter", "unzweifelhaftes", "usw", "usw.", "vergangen", "vergangene", "vergangener", "vergangenes", "vermag", "vermögen", "vermutlich", "vermutliche", "vermutlichem", "vermutlichen", "vermutlicher", "vermutliches", "veröffentlichen", "veröffentlicher", "veröffentlicht", "veröffentlichte", "veröffentlichten", "veröffentlichtes", "verrate", "verraten", "verriet", "verrieten", "version", "versorge", "versorgen", "versorgt", "versorgte", "versorgten", "versorgtes", "viel", "viele", "vielen", "vieler", "vielerlei", "vieles", "vielleicht", "vielmalig", "vielmals", "vier", "voellig", "voellige", "voelligem", "voelligen", "voelliger", "voelliges", "voelligst", "vollends", "völlig", "völlige", "völligem", "völligen", "völliger", "völliges", "völligst", "vollstaendig", "vollstaendige", "vollstaendigem", "vollstaendigen", "vollstaendiger", "vollstaendiges", "vollständig", "vollständige", "vollständigem", "vollständigen", "vollständiger", "vollständiges", "vom", "von", "vor", "voran", "vorbei", "vorgestern", "vorher", "vorherig", "vorherige", "vorherigem", "vorheriger", "vorne", "vorüber", "vorueber", "wachen", "waehrend", "waehrenddessen", "waere", "während", "währenddessen", "wann", "war", "wär", "wäre", "waren", "wären", "warst", "wart", "warum", "was", "weder", "weg", "wegen", "weil", "weiß", "weit", "weiter", "weitere", "weiterem", "weiteren", "weiterer", "weiteres", "weiterhin", "weitestgehend", "weitestgehende", "weitestgehendem", "weitestgehenden", "weitestgehender", "weitestgehendes", "weitgehend", "weitgehende", "weitgehendem", "weitgehenden", "weitgehender", "weitgehendes", "welche", "welchem", "welchen", "welcher", "welches", "wem", "wen", "wenig", "wenige", "weniger", "wenigstens", "wenn", "wenngleich", "wer", "werde", "werden", "werdet", "weshalb", "wessen", "weswegen", "wichtig", "wie", "wieder", "wiederum", "wieso", "wieviel", "wieviele", "wievieler", "wiewohl", "will", "willst", "wir", "wird", "wirklich", "wirklichem", "wirklicher", "wirkliches", "wirst", "wo", "wobei", "wodurch", "wofuer", "wofür", "wogegen", "woher", "wohin", "wohingegen", "wohl", "wohlgemerkt", "wohlweislich", "wolle", "wollen", "wollt", "wollte", "wollten", "wolltest", "wolltet", "womit", "womoeglich", "womoegliche", "womoeglichem", "womoeglichen", "womoeglicher", "womoegliches", "womöglich", "womögliche", "womöglichem", "womöglichen", "womöglicher", "womögliches", "woran", "woraufhin", "woraus", "worden", "worin", "wuerde", "wuerden", "wuerdest", "wuerdet", "wurde", "würde", "wurden", "würden", "wurdest", "würdest", "wurdet", "würdet", "www", "x", "z.B.", "zahlreich", "zahlreichem", "zahlreicher", "zB", "zb.", "zehn", "zeitweise", "zeitweisem", "zeitweisen", "zeitweiser", "ziehen", "zieht", "ziemlich", "ziemliche", "ziemlichem", "ziemlichen", "ziemlicher", "ziemliches", "zirka", "zog", "zogen", "zu", "zudem", "zuerst", "zufolge", "zugleich", "zuletzt", "zum", "zumal", "zumeist", "zumindest", "zunächst", "zunaechst", "zur", "zurück", "zurueck", "zusammen", "zusehends", "zuviel", "zuviele", "zuvieler", "zuweilen", "zwanzig", "zwar", "zwei", "zweifelsfrei", "zweifelsfreie", "zweifelsfreiem", "zweifelsfreien", "zweifelsfreier", "zweifelsfreies", "zwischen", "zwölf"]

stopwords_en = list(set(stopwords_en_1 + stopwords_en_2))
stopwords_de = list(set(stopwords_de_1 + stopwords_de_2))

all_stopwords = list(set(stopwords_en + stopwords_de))
all_stopwords = [word.lower() for word in all_stopwords]

# import pickle

# filename = 'stopwords_xxx.pkl'
# with open(filename, 'wb') as fp:
#     pickle.dump(all_stopwords, fp)

In [20]:
fuzz.ratio('5G network', '5G 4G network')

87

In [16]:
def is_numeric(nc):
    
    for term in nc.split():
        term = re.sub(r'[^\w\s]',' ',term)
        if term.isnumeric():
            return False
    return True

def clean_normalized_word(nc):
    
    clean_words = []
    for term in nc.split():
        if term.lower() not in all_stopwords:
            clean_words.append(term)
    return " ".join(clean_words)

def get_shorter_text(phrase_1, phrase_2):
    
    if len(phrase_1) < len(phrase_2):
        return phrase_1
    else:
        return phrase_2

def remove_smallcase_words(noun_chunk):
    
    word_list = []
    for word in noun_chunk.split():
        last_char = word[0]
        if last_char.isupper():
            word_list.append(word)
                        
    return ' '.join(word_list)

def get_cleaned_noun_chunks(noun_chunks):
    
    original_nc = [nc[0] for nc in noun_chunks]
    normalized_nc = [nc[1] for nc in noun_chunks]
    
    noun_chunks = normalized_nc
    
    noun_chunks = list(set(noun_chunks))
    remove_phrases = [] 
    phrases_len = len(noun_chunks)
    
    for idx_1 in range(phrases_len):

        phrase_1 = noun_chunks[idx_1]
        for idx_2 in range(idx_1 + 1, phrases_len):
            phrase_2 = noun_chunks[idx_2]

            if fuzz.ratio(phrase_1, phrase_2) > 85:
                remove_phrases.append(get_shorter_text(phrase_1, phrase_2))

    final_noun_chunks = list(set(noun_chunks) - set(remove_phrases))
    
    normalized_nc_idx = [normalized_nc.index(nc) for nc in final_noun_chunks]
    original_nc_updated = [original_nc[idx] for idx in normalized_nc_idx]
    
    final_noun_chunks = [(nc1, nc2) for nc1, nc2 in zip(original_nc_updated, final_noun_chunks)]
    
    return final_noun_chunks

def clean_string_from_puntuation_and_special_chars(s):
    invalidcharacters = set(string.punctuation)
    if any(char in invalidcharacters for char in s):
        s_ = s.translate(str.maketrans('', '', string.punctuation))
    else:
        s_ = s
    return s_

def remove_undesired_words(nc, language_flag):
    nc_splitted = nc.split()
    if language_flag == "EN":
        irrelevant_determiners_and_words = \
        ["the","an", "a", "one", "on", "at", "by", "s"]
    else:
        irrelevant_determiners_and_words = \
        ["system","möglichkeit","wesentlichen", "rollen", "der", "die", "selbst", "fähig", "das", "dem", "den", "ein", "eine", "des", "bei", "muss", "sich", "eines", "s", "es", "einen", "einem"]
    cleaned_nc = nc
    
    if len(cleaned_nc.split()) >= 1:
        if nc_splitted[0].lower() in irrelevant_determiners_and_words:
            cleaned_nc = ' '.join([w for w in nc_splitted[1:]])
    if len(cleaned_nc.split()) >= 1:
        if cleaned_nc.split()[-1].lower() in irrelevant_determiners_and_words:
            cleaned_nc = ' '.join([w for w in cleaned_nc.split()[:-1]])
        if len(cleaned_nc.split()) >= 1:
            if cleaned_nc.split()[-1].lower() in irrelevant_determiners_and_words:
                cleaned_nc = ' '.join([w for w in cleaned_nc.split()[:-1]])
    if len(cleaned_nc.split()) >= 1:
        if cleaned_nc.split()[0].lower() in irrelevant_determiners_and_words:
            cleaned_nc = ' '.join([w for w in cleaned_nc.split()[1:]])            
    return cleaned_nc.strip()

def get_syntactically_nested_ncs(sent, boundary):
    terms_to_be_removed = set()
    nested_terms_short = set()
    nested_terms_longs = set()
    rule_for_longs = r"({0})(.*?)({0})(.*?)({1})(.*?)({1})".format(boundary[0], boundary[1])
    rule_for_shorts = r"({0})(.*?)({1})".format(boundary[0], boundary[1])
    results_for_longs = re.finditer(rule_for_longs, sent)
    results_for_shorts = re.finditer(rule_for_shorts, sent)
    for match in results_for_longs:
        nested_terms_longs.add((sent[match.start():match.end()]))
    for match in results_for_shorts:
        nested_terms_short.add((sent[match.start():match.end()]))
    for s in nested_terms_short:
        for l in nested_terms_longs:
            if s in l:
                terms_to_be_removed.add(s)
    pre_result_with_possible_redundancies = nested_terms_short.union(nested_terms_longs)
    final_result = pre_result_with_possible_redundancies.difference(terms_to_be_removed)
    return final_result

def extract_and_clean_ncs_raat(sent, language_flag, representation_level):
    if representation_level == "TERM":
        return [(sent, normalize_nc(sent, language_flag))], []
    else:
        noun_chunks_set_for_sent = set()
        if language_flag == "EN":
            doc = nlp_en(sent)
            determiners_list = ["the"]
            noun_chunks_extracted_with_patterns = set([t.text for t in doc.noun_chunks])
            noun_chunks_with_bounds = set()
        else:
            determiners_list = ["der", "die", "das"]
            noun_chunks_in_quotes = get_syntactically_nested_ncs(sent, boundary=["\"", "\""])
            noun_chunks_with_bounds = noun_chunks_in_quotes.union(get_syntactically_nested_ncs(sent, boundary=["\(", "\)"]))
            noun_chunks_extracted_with_patterns = set(get_ncs(sent, pattern1)).union(set(get_ncs(sent, pattern2)))


        for chunk in noun_chunks_extracted_with_patterns:
            tmp_1 = chunk
            if len(tmp_1.split()) <= 1 and tmp_1.strip().lower() in determiners_list:
                pass
            else:
                nc_with_removed_determiners = remove_undesired_words(tmp_1, language_flag)
                if len(clean_string_from_puntuation_and_special_chars(nc_with_removed_determiners))>0:
                    noun_chunks_set_for_sent.add(nc_with_removed_determiners)

        composed_terms = set()
        terms_to_be_removed = set()
        for nc1 in noun_chunks_set_for_sent:
            for nc2 in noun_chunks_set_for_sent:
                if nc1 in nc2 and len(nc1)<len(nc2):
                    terms_to_be_removed.add(nc1)
                if language_flag == "EN":
                    comp_term1 = nc1 + " of " + nc2
                    comp_term2 = nc1 + " and " + nc2
                    if comp_term1 in sent:
                        composed_terms.add(comp_term1)
                    if comp_term2 in sent:
                        composed_terms.add(comp_term2)
                else:
                    conj = [" und "," und -", "- und "," /-", " / -", " / ", " für ", " "]
                    for con in conj:
                        comp_term = nc1 + con + nc2
                        if comp_term in sent:
                            composed_terms.add(comp_term)
                            if con in [" und -", "- und ", " /-", " / -", " / ", " "]:
                                terms_to_be_removed.add(nc1)
                                terms_to_be_removed.add(nc2)
                            if "-" == nc1[-1] or "-" == nc2[0]:
                                terms_to_be_removed.add(nc1)
                                terms_to_be_removed.add(nc2)                                
            if len(nc1) >= 4 and "muss" in nc1.split()[1:-1]:
                terms_to_be_removed.add(nc1)
                composed_terms_tuple_split = nc1.split(" muss ")
                composed_terms.add(composed_terms_tuple_split[0])
                composed_terms.add(composed_terms_tuple_split[1])
            if nc1 + "/s" in sent:
                terms_to_be_removed.add(nc1)
                composed_terms.add(nc1 + "/s" )
            if len(nc1) <= 2:
                terms_to_be_removed.add(nc1)
                
        noun_chunks_set_for_sent = noun_chunks_set_for_sent.difference(terms_to_be_removed)   
        
        for nc1 in noun_chunks_set_for_sent:
            for nc2 in noun_chunks_with_bounds:
                if nc1 + " " + nc2 in sent:
                    composed_terms.add(nc1 + " " + nc2)
        
            
        found_terms = noun_chunks_set_for_sent.union(composed_terms)

        cleaned_terms = []
        set_of_compositas = set()
        for original_term in found_terms:
            if len(original_term) > 0:
                cleaned_terms.append((original_term, normalize_nc(original_term, language_flag)))
        return list(set(cleaned_terms)), list(set_of_compositas)
    
def extract_and_clean_ncs(sent, language_flag, representation_level): #TERM, TEXT
    language_flag = language_flag.upper()
    
    if representation_level == "TERM":
        return [(sent, normalize_nc(sent, language_flag))]
    else:
        noun_chunks_set_for_sent = set()
        if language_flag == "EN":
            doc = nlp_en(sent)
            determiners_list = ["each", "the", "an", "each", "them", "therefore", "e.g.", ",", "a", "one", "on", "at",
                                "by", "when", "such", "all", "this", "given", "its", "their", "as"]
        else:
            doc = nlp_de(sent)
            determiners_list = ["der", "die", "das", "dem", "den", "ein", "eine", "des", "bei", "beim"]
        for chunk in doc.noun_chunks:
            tmp_1 = chunk.text
            tmp_1 = tmp_1.strip("(")
            tmp_1 = tmp_1.strip(")")
            if len(tmp_1.split()) == 1 and tmp_1.strip().lower() in determiners_list:
                pass
            else:
                nc_with_removed_determiners = remove_determiners(tmp_1, language_flag)
                if nc_with_removed_determiners is not None and len(clean_string_from_puntuation_and_special_chars(nc_with_removed_determiners)) > 0:
                    noun_chunks_set_for_sent.add(nc_with_removed_determiners)

        composed_terms = set()
        for nc1 in noun_chunks_set_for_sent:
            for nc2 in noun_chunks_set_for_sent:
                if language_flag == "EN":
                    comp_term1 = nc1 + " of " + nc2
                    comp_term2 = nc1 + " and " + nc2
                    comp_term3 = nc1 + "'s " + nc2
                    if comp_term1 in sent:
                        composed_terms.add(comp_term1)
                    if comp_term2 in sent:
                        composed_terms.add(comp_term2)
                    if comp_term3 in sent:
                        composed_terms.add(comp_term3)
                else:
                    comp_term = nc1 + " und " + nc2
                    if comp_term in sent:
                        composed_terms.add(comp_term)
        found_terms = noun_chunks_set_for_sent.union(composed_terms)

        cleaned_terms = []
        for original_term in found_terms:
            
            if len(original_term.split()) < 4 and original_term.lower() not in all_stopwords and is_numeric(original_term):
                if len(original_term) > 0:
                    normalized_word = normalize_nc(original_term, language_flag)
                    normalized_word = re.sub(r'[^\w\s]',' ',normalized_word)
                    normalized_word = clean_normalized_word(normalized_word)
                    
                    if language_flag == "DE" and len(normalized_word) > 1:
                        normalized_word = remove_smallcase_words(normalized_word)
                    
                    if len(normalized_word) > 0 and is_numeric(normalized_word):
                        cleaned_terms.append((original_term, normalized_word))
                    
#         cleaned_terms = get_cleaned_noun_chunks(cleaned_terms)
        return cleaned_terms

In [17]:
text = """As cellular communication has progressed in the last two decades, we’ve rapidly approached the theoretical limits for wireless data transmission set by Shannon’s Law. Every successive cellular generation has brought dramatic increases in data rates. 2G networks offered a maximum theoretical data rate of 40 kbps–but today’s 4G LTE-Advanced networks have peak theoretical data rates of 1 Gbps . 5G takes that one step further; next-generation networks will have peak theoretical data rates of 20 Gbps for downlink and 10 Gbps for uplink . Theoretical peak rates are just that: theoretical . You probably don’t see 1 Gbps download speeds on your LTE Android or iPhone handset. The more useful metric defined by the International Telecommunications Union (ITU) for the IMT-2020 standard (basically the 5G standard) is user experience data rate , which is the data rate experienced by users in at least 95% of the locations where the network is deployed for at least 95% of the time. By this measure, at a minimum of 100 Mbps , 5G should be at least five times faster than average 4G speeds . To understand how 5G achieves these higher data rates, we need to dig into Shannon’s Law to see how engineers have tackled each of the limiting factors from previous generations. Please note, we’re completely ignoring latency here. Latency, or the time it takes to reach a server, is not limited by Shannon’s Law and has a huge impact on everyday Internet usage. We’ll cover how 5G networks improve latency in a future post."""

noun_chunks_raat = extract_and_clean_ncs(text, language_flag='EN', representation_level='TEXT')
noun_chunks_raat

[('Latency', 'latency'),
 ('dramatic increases', 'dramatic increase'),
 ('future post', 'future post'),
 ('locations', 'location'),
 ('2G networks', '2G network'),
 ('huge impact', 'huge impact'),
 ('5G networks', '5G network'),
 ('last two decades', 'decade'),
 ('cellular communication', 'cellular communication'),
 ('previous generations', 'previous generation'),
 ('limiting factors', 'limiting factor'),
 ('server', 'server'),
 ('5G', '5G'),
 ('theoretical data rates', 'theoretical data rate'),
 ('measure', 'measure'),
 ('everyday Internet usage', 'everyday Internet usage'),
 ('users', 'user'),
 ('downlink', 'downlink'),
 ('theoretical limits', 'theoretical limit'),
 ('how 5G', '5G'),
 ('minimum', 'minimum'),
 ('that one step', 'step'),
 ('wireless data transmission', 'wireless data transmission'),
 ('time', 'time'),
 ('Shannon’s Law', 'Shannon Law'),
 ('average 4G speeds', 'average 4G speed'),
 ('more useful metric', 'metric'),
 ('data rate', 'data rate'),
 ('uplink', 'uplink'),
 ('y

In [18]:
text_de = """Seit mehr als 35 Jahren bürgt der Name SAP (Systeme, Anwendungen, Produkte in der Datenverarbeitung) für Innovation, Erfolg und Kreativität. Als führender Anbieter von Unternehmenssoftware und drittgrößter unabhängiger Softwarelieferant der Welt entwickeln wir maßgeschneiderte Unternehmenslösungen für mehr als 46.100 Kunden rund um den Globus. Unseren Erfolg verdanken wir der hohen Qualität unserer Produkte sowie der langjährigen Erfahrung und dem Know-how unserer mehr als 43.800 Mitarbeiter weltweit.Mit Niederlassungen in mehr als 50 Ländern erzielte die SAP im Geschäftsjahr 2007 einen vorläufigen Umsatz von 10,3 Milliarden Euro. Die SAP AG ist an verschiedenen Börsen einschließlich der Frankfurter Wertpapierbörse und der New Yorker Wertpapierbörse (NYSE) unter dem Tickersymbol "SAP" gelistet.Das Lösungsportfolio von SAP ebnet den Weg zu einer reibungslosen, standort- und zeitunabhängigen Zusammenarbeit zwischen Kunden, Partnern und Mitarbeitern. Ob Customer Relationship Management, Supply Chain Management oder Product Lifecycle Management: SAP-Lösungen optimieren all jene zentralen Geschäftsprozesse, die heute für den Erfolg entscheidend sind."""

extract_and_clean_ncs(text_de, language_flag='DE', representation_level='TEXT')

[('unserer Produkte', 'Produkt'),
 ('Partnern', 'Partner'),
 ('Frankfurter Wertpapierbörse', 'Frankfurter Wertpapierbörse'),
 ('Unseren Erfolg', 'Erfolg'),
 ('einen vorläufigen Umsatz', 'Umsatz'),
 ('New Yorker Wertpapierbörse', 'Yorker Wertpapierbörse'),
 ('langjährigen Erfahrung', 'Erfahrung'),
 ('Tickersymbol "SAP', 'Tickersymbol SAP'),
 ('Partnern und Mitarbeitern', 'Partnern Mitarbeiter'),
 ('Produkte', 'Produkt'),
 ('Name SAP', 'SAP'),
 ('Niederlassungen', 'Niederlassung'),
 ('10,3 Milliarden Euro', 'Milliarden Euro'),
 ('SAP-Lösungen', 'SAP Lösungen'),
 ('verschiedenen Börsen', 'Börse'),
 ('Anwendungen', 'Anwendung'),
 ('NYSE', 'NYSE'),
 ('Lösungsportfolio', 'Lösungsportfolio'),
 ('Product Lifecycle Management', 'Product Lifecycle Management'),
 ('Erfolg', 'Erfolg'),
 ('Innovation', 'Innovation'),
 ('Kunden', 'Kunde'),
 ('maßgeschneiderte Unternehmenslösungen', 'Unternehmenslösungen'),
 ('SAP', 'SAP'),
 ('Mitarbeitern', 'Mitarbeiter'),
 ('Welt', 'Welt'),
 ('führender Anbieter', 

In [8]:
final_df = pd.read_pickle(os.getcwd()+'/../dataframes/final_dataframe.pkl')
final_df.sample(2)

,id,text,text_len,lang,text_tokens,nc_vec,label,label_name,title,pubDate,url
787,210705_news_362332,\n A simulation program that can be install...,755,en,"[a simulation program, that, ordinary computer...","[[-0.009062029421329498, -0.002376047661527991...",1,technology,Ultra-Low Cost Simulation Program Augments Pil...,2020-12-08 16:52:00,https://www.defense.gov/Explore/News/Article/A...
9234,210705_news_61693,A nimated features have been big business si...,1805,en,"[a nimated feature, big business, Walt Disney’...","[[-0.009228140115737915, -0.013340845704078674...",1,technology,The Surprise and Wonder of Early Animation,2019-12-24 16:24:42,https://www.newyorker.com/magazine/2019/12/30/...


In [17]:
final_df[final_df['id'] == '210705_news_460007']

,id,text,text_len,lang,text_tokens,nc_vec,label,label_name,title,pubDate,url
677,210705_news_460007,Frink What's New *\n FAQ *\n Download ...,70863,en,"[what, New *\n FAQ *\n Download, Frink...","[[0.0008865785785019398, -0.000791810860391706...",1,technology,Frink,2021-03-31 09:32:07,https://futureboy.us/frinkdocs/


In [22]:
len(text)

478034

In [23]:
len(text.split())

70863

In [21]:
text = final_df[final_df['id'] == '210705_news_460007'].text.values[0]
lang = final_df[final_df['id'] == '210705_news_460007'].lang.values[0]

extract_and_clean_ncs(text, language_flag=lang, representation_level='TEXT')

KeyboardInterrupt: 

In [ ]:
cdd_df = pd.read_pickle(os.getcwd()+'/../dataframes/query_dataframes_cdd_backup/retrieval_dataset_mr_cdd.pkl')
cdd_df.sample(2)

In [ ]:
len(cdd_df.index)

In [ ]:
cdd_df['noun_chunks_xxx'] = cdd_df.apply(lambda x:extract_and_clean_ncs(x['text'], x['lang'], 'TEXT'), axis=1)

In [ ]:
cdd_df.noun_chunks_xxx.values[0]

In [ ]:
combined_noun_chunks = []

for page_id, nc_list in zip(cdd_df.page_id.values, cdd_df.noun_chunks_xxx.values):
    combined_noun_chunks.extend(nc_list)
    
#     with open(page_id+".txt", "w", newline="", encoding="utf-8") as f:
#         writer = csv.writer(f)
#         writer.writerows(nc_list)

In [ ]:
combined_noun_chunks = list(set(combined_noun_chunks))
sorted_nc = sorted(combined_noun_chunks, key=lambda tup: tup[0])

In [ ]:
len(sorted_nc)

In [ ]:
with open("noun_chunks_sorted.txt", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(sorted_nc)

In [ ]:
pspell = phunspell.Phunspell('de_DE')

In [ ]:
pspell.lookup('Sicherheit')

In [ ]:
h.spell('items')

# Noun-chunk cleaning analysis


1. Characters:

    - Δ,Δ
    - α,α
    - μ,μ
    - τ,τ
    - ϵ,ϵ
    
2. German compositer -- only when it is a label

    - Obsoleszenzbeseitigung
    - Aufklärungsmanagements
    - Bearbeitungsmöglichkeiten
    
3. Newline, numeric, currency ....

    - 26(12,26 12
    - 2m sales milestone,2m sales milestone
    - "30,000 square meters",30 000 square meter
    - "300,000 new jobs",300 000 job
    - 32%,32
    - "32,6 Millionen Euro",Millionen Euro
    - 34%,34
    - 34%-48%,34 48
    - 36% to 44%,36 44
    - 36(12,36 12
    - 4%,4